In [3]:
import os
import shutil
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow 
import keras
from tensorflow.keras.models import load_model
import customtkinter as ctk
import os


# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64
 
# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 20

CLASSES_LIST = ["fight","noFight"]
FIRE_CLASSES_LIST = ["Fire","noFire"]

model_path = ''
file_path = ''

def predict_fire__video(video_file_path):
    model=load_model(model_path)
    video_reader = cv2.VideoCapture(video_file_path)
 
    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
    # Declare a list to store video frames we will extract.
    frames_list = []
    
    # Store the predicted class in the video.
    predicted_class_name = ''
 
    # Get the number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
 
    # Calculate the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)
 
    # Iterating the number of times equal to the fixed length of sequence.
    for frame_counter in range(SEQUENCE_LENGTH):
 
        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
 
        success, frame = video_reader.read() 
 
        if not success:
            break
 
        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Normalize the resized frame.
        normalized_frame = resized_frame / 255
        
        # Appending the pre-processed frame into the frames list
        frames_list.append(normalized_frame)
 
    predicted_labels_probabilities = []
    predicted_label_lst = []
    # Passing the  pre-processed frames to the model and get the predicted probabilities.
    avg = [0,0]
    for i in range(SEQUENCE_LENGTH):
        
        predicted_labels_probabilities.append(model.predict(np.expand_dims(frames_list[i], axis = 0))[0])
        avg[0] += predicted_labels_probabilities[len(predicted_labels_probabilities)-1][0]
        avg[1] += predicted_labels_probabilities[len(predicted_labels_probabilities)-1][1]
    
    avg[0] /= 20
    avg[1] /= 20
    if avg[0] > avg[1]:
        predicted_label = 0
    else:
        predicted_label = 1
    
    # Get the class name using the retrieved index.
    predicted_class_name = FIRE_CLASSES_LIST[predicted_label]
    
    # Display the predicted class along with the prediction confidence.
    print(f'Predicted: {predicted_class_name}\nConfidence: {avg[predicted_label]}')
        
    confidence = str(predicted_labels_probabilities[predicted_label])
    print(f'Predicted: {predicted_class_name}\nConfidence: {confidence}')
    Result = 'Predicted: ' + predicted_class_name + ' and Confidence: ' +str(avg[predicted_label]) 

    # Create a new label to display the result
    result_label.configure(text=Result)

    video_reader.release()


def model_with_transfer_learning():
    global model_path
    global file_path
    global SEQUENCE_LENGTH
    model_path = '../Resources/Models/With Transfer Learning.h5'
    SEQUENCE_LENGTH=20
    predict_video(file_path)

def without_transfer_learning():
    global SEQUENCE_LENGTH
    global model_path
    global file_path
    SEQUENCE_LENGTH=20
    model_path = '../Resources/Models/Without Transfer Learning.h5'
    predict_video(file_path)

def optic_model_with_transfer_learning():
    global SEQUENCE_LENGTH
    global model_path
    global file_path
    SEQUENCE_LENGTH=19
    model_path = '../Resources/Models/Optic With Transfer Learning.h5'
    predict_video(file_path)

def optic_model_without_transfer_learning():
    global SEQUENCE_LENGTH
    global model_path
    global file_path
    SEQUENCE_LENGTH=19
    model_path = '../Resources/Models/Optic Without Transfer Learning.h5'
    predict_video(file_path)


def fire_model():
    global model_path
    global file_path
    model_path='../Resources/Models/fire_model_84.h5'
    predict_fire__video(file_path)

def predict_video(video_file_path):
    global model_path
    #pre trained model path, models available on google drive
    MoBiLSTM_model=load_model(model_path) 
    video_reader = cv2.VideoCapture(video_file_path)
 
    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
    # Declare a list to store video frames we will extract.
    frames_list = []
    
    # Store the predicted class in the video.
    predicted_class_name = ''
 
    # Get the number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
 
    # Calculate the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)
 
    # Iterating the number of times equal to the fixed length of sequence.
    for frame_counter in range(SEQUENCE_LENGTH):
 
        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
 
        success, frame = video_reader.read() 
 
        if not success:
            break
 
        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Normalize the resized frame.
        normalized_frame = resized_frame / 255
        
        # Appending the pre-processed frame into the frames list
        frames_list.append(normalized_frame)
 
    # Passing the  pre-processed frames to the model and get the predicted probabilities.
    predicted_labels_probabilities = MoBiLSTM_model.predict(np.expand_dims(frames_list, axis = 0))[0]
 
    # Get the index of class with highest probability.
    predicted_label = np.argmax(predicted_labels_probabilities)
 
    # Get the class name using the retrieved index.
    predicted_class_name = CLASSES_LIST[predicted_label]

    
    confidence = str(predicted_labels_probabilities[predicted_label])
    print(f'Predicted: {predicted_class_name}\nConfidence: {confidence}')
    Result = 'Predicted: ' + predicted_class_name + ' and Confidence: ' + confidence

    # Create a new label to display the result
    result_label.configure(text=Result)

    video_reader.release()


def button_clicked():
    global file_path
    Result =''
    file_path = ctk.filedialog.askopenfilename()
    if file_path:
        file_name = os.path.basename(file_path)
        label.configure(text="Selected File: " + file_name)
        button_upload.pack_forget()  # Hide the "Upload" button
        result_label.pack(pady=(30, 0))
        fight_model1.pack(side="left", padx=10, pady=(0, 10))  
        fight_model2.pack(side="left", padx=10, pady=(0, 10))
        optic_without_transfer.pack(side="left", padx=10, pady=(0, 10))  
        optic_with_transfer.pack(side="left", padx=10, pady=(0, 10))
        fire_model.pack(side="left", padx=10, pady=(0, 10))  
     
    else:
        label.configure(text="No file selected")

def apply_model():
    print("Applying model...")  

# Create a new tkinter window
window = ctk.CTk()

window.title("Intelligent Surveillance System")

window.minsize(width=400, height=300)

window.resizable(True, True)  

label = ctk.CTkLabel(window, text="No file selected")
label.pack(pady=(20, 0))

button_upload = ctk.CTkButton(window, text="Upload", command=button_clicked)
button_upload.pack(pady=10)

result_label = ctk.CTkLabel(window, text="Prediction and confidence will be Displayed ")
result_label.pack(pady=(30, 0))
result_label.pack_forget()

fight_model1 = ctk.CTkButton(window, text="Transfer Learning", command=model_with_transfer_learning)
fight_model1.pack(side="left", padx=10)  
fight_model1.pack_forget()  

fight_model2 = ctk.CTkButton(window, text="Without Transfer Learning", command=without_transfer_learning)
fight_model2.pack(side="left", padx=10, pady=10) 
fight_model2.pack_forget()

fire_model = ctk.CTkButton(window, text="Apply Fire Model", command=fire_model)
fire_model.pack(side="left", padx=10, pady=10)  
fire_model.pack_forget()

optic_without_transfer = ctk.CTkButton(window, text="Optic Without Transfer Learning", command=optic_model_without_transfer_learning)
optic_without_transfer.pack(side="left", padx=10)  
optic_without_transfer.pack_forget()  

optic_with_transfer = ctk.CTkButton(window, text="Optic with Transfer Learning", command=optic_model_with_transfer_learning)
optic_with_transfer.pack(side="left", padx=10, pady=10) 
optic_with_transfer.pack_forget()

window.mainloop()


1/1 [==============================] - 0s 24ms/step
Predicted: noFire
Confidence: 0.9999998867511749
Predicted: noFire
Confidence: [3.234658e-08 1.000000e+00]
